## Forward kinematics of a 1dof planar robot

### Case 1) Tool frame directions aligned with base frame at reference configuration
<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-1dof-revolute.png?raw=true" width=400 />

In [1]:
import numpy as np
import doctest

In [25]:
def fwd_kinematics1(th1, l1=2):
    '''
    Implements the forward kinematics of a planar robot with one revolute joint. 
    
    Arguments
    ---------
    th1    : float
       Angle in radians.
    l1     : float
       Length of the link.
 
    Returns
    -------
    x : float
       The position in the global x-direction of the end-effector (tool point)
    y : float
       The position in the global y-direction of the end-effector (tool point)
    theta : float
       The orientation of the end-effector with respect to the positive global x-axis.
       The angle returned is in the range [-np.pi, np.pi]
    
    Tests
    ------
    1) End-effector pose at reference position
    >>> x, y, th = fwd_kinematics1(0)
    >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, th)
    '(2.00, 0.00, 0.00)'
    
    2) End-effector pose at 90 degrees in joint
    >>> x, y, th = fwd_kinematics1(np.pi/2)
    >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, th)
    '(0.00, 2.00, 1.57)'
    
    3) Angle of end-effector should be in range [-pi, pi]
    >>> x, y, th = fwd_kinematics1(3*np.pi/2)
    >>> "%0.2f" %(th)
    '-1.57'
    
    4) End-effector is always at constant distance from the origin
    >>> poses = [fwd_kinematics1(th1_, 3)
    ... for th1_ in np.arange(0, 2*np.pi, 0.2) ] 
    >>> distances = np.array([np.sqrt(x_**2 + y_**2) for x_, y_, th_ in poses])
    >>> np.any(np.abs(distances - 3) > 1e-12)
    False
    
    5) End-effector velocity vector at 45 degrees in angle, using finite difference, 
    link rotating at unit speed (1 rad/s)
    >>> delta_angle = 1e-4
    >>> th_1 = np.pi/4
    >>> l_1 = 2.5
    >>> x0, y0, th0 = fwd_kinematics1(th_1, l_1)
    >>> x1, y1, th1 = fwd_kinematics1(th_1+delta_angle, l_1)
    >>> vx = (x1-x0)/delta_angle
    >>> vy = (y1-y0)/delta_angle
    >>> vx_true = -l_1*np.sin(th_1)
    >>> vy_true = l_1*np.cos(th_1)
    >>> abs(vx-vx_true) < 1e-3
    True
    >>> abs(vy-vy_true) < 1e-3
    True
    
    6 ) NOW IT IS YOUR TURN. WRITE A TEST THAT SHOULD PASS IF THE IMPLEMENTATION 
    IS CORRECT
    >>> my_own_test
    '''
    x = 0
    y = 0
    theta = 0
    
    # make theta be in [-np.pi, np.pi]
    if theta > np.pi:
        theta -= 2*np.pi
    elif theta < -np.pi:
        theta += 2*np.pi
        
    return (x, y, theta)


#### Run doctests
If tests pass, no output is generated.

In [26]:
doctest.run_docstring_examples(fwd_kinematics1, globals())

### Case 2) Tool frame rotated wrt base frame at reference configuration
<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-1dof-revolute-rotated-tool.png?raw=true" width=400 />

In [1]:
import numpy as np
import doctest

In [37]:
def fwd_kinematics2(th1, l1=2, le=0.5):
    '''
    Implements the forward kinematics of a planar robot with one revolute joint,
    with tool-frame rotated with respect to the base frame. The tool frame is
    rotated 90 degrees so that in the reference configuration, 
    the tool x-vector points in the direction of the base y-vector, and the
    tool y-vector points in the opposite direction of the base x-vector. 
    
    Arguments
    ---------
    th1    : float
       Angle in radians.
    l1     : float
       Length of the link.
    le      : Offset of the tool center point
    Returns
    -------
    tcp : nd-array (x,y,z)
       The position of the tool center point (the origin of the tool frame) 
       in the base frame.
    theta : float
       The orientation of the end-effector with respect to the positive global x-axis.
       The angle returned is in the range [-np.pi, np.pi]
    
    Tests
    ------
    1) End-effector pose at reference position
    >>> p, th = fwd_kinematics2(0)
    >>> "x=%0.2f, y=%0.2f, z=%0.2f, th=%0.2f" %(p[0], p[1], p[2], th)
    'x=2.00, y=0.50, z=0.00, th=1.57'
    
    2) End-effector pose at 60 degrees in joint
    >>> p, th = fwd_kinematics2(np.pi/3)
    >>> "x=%0.2f, y=%0.2f, z=%0.2f, th=%0.2f" %(p[0], p[1], p[2], th)
    'x=0.57, y=1.98, z=0.00, th=2.62'
    
    3) Angle of end-effector should be in range [-pi, pi]
    >>> p, th = fwd_kinematics2(3*np.pi/2)
    >>> "%0.2f" %(np.abs(th))
    '0.00'
    
    4) End-effector is always at constant distance from the origin
    >>> poses = [fwd_kinematics2(th1_, l1=3, le=1)
    ... for th1_ in np.arange(0, 2*np.pi, 0.2) ] 
    >>> distances = np.array([np.sqrt(p_[0]**2 + p_[1]**2) for p_, th_ in poses])
    >>> np.any(np.abs(distances - np.sqrt(10)) > 1e-12)
    False
    '''
    
    tcp0 = np.array([l1, le, 0]) # TCP in base frame at ref config
    R = np.eye(3)
    
    tcp = np.dot(R, tcp0)
    theta = 0
    
    return (tcp, theta)


#### Run doctests
If tests pass, no output is generated.

In [38]:
doctest.run_docstring_examples(fwd_kinematics2, globals())

# The spatial and body Jacobians
Consider the case 1) above
<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-1dof-revolute.png?raw=true" width=400 />
The forward kinematics map $$^Sp = f(\theta)$$ gives the position of the tcp in the spatial frame as a function of the angle $\theta$. This was solved above. We are also interested in the orientation of the tool, of course. The rotation matrix that expresses the orientation of the tool frame w.r.t. the spatial frame evidently also depends on the angle $\theta$. We write it $R_{ts}(\theta)$. The subscript $ts$ indicates that it will operate on vectors expressed in the spatial frame, and return that vector expressed in the tool frame. For instance, the spatial x-vector $$ ^Sx_S = \begin{bmatrix}1\\0\\0\end{bmatrix}$$ is in the tool frame given by 
$$ ^Bx_S = R_{ts}(\theta)\,^Sx_S = R_{ts}(\theta)\, \begin{bmatrix}1\\0\\0\end{bmatrix}, $$
which is the first column of $R_{ts}(\theta)$. 

We can find the velocity of the tcp by using the chain rule
$$^S v = \frac{d}{dt} ^Sp = \underbrace{\frac{d}{d\theta} f(\theta)}_{J_s} \frac{d}{dt}\theta, $$
where $$J_s(\theta) = \frac{d}{d\theta}f(\theta)$$ is referred to as the *spatial Jacobian*. Multiplying the spatial jacobian with the angular velocity gives us the velocity of the TCP in the static, spatial frame. 

Often it is also of interest to know the velocity of the TCP (with respect to the spatial frame), expressed in the tool frame itself. This can be found by noting that
$$ ^Tv = R_{ts}(\theta) ^Sv = \underbrace{R_{ts}(\theta)\frac{d}{d\theta}f(\theta)}_{J_b} \frac{d}{dt}\theta, $$
where $$J_b(\theta) = R_{ts}(\theta)\frac{d}{d\theta}f(\theta)$$ is referred to as the *body Jacobian*.


In [1]:
import numpy as np
import doctest

In [77]:
def fwd_kinematics3(th1, l1=2):
    '''
    Implements the forward kinematics of a planar robot with one revolute joint,
    where the tool frame is aligned with the spatial frame at the reference configuration.
    
    Arguments
    ---------
    th1    : float
       Angle in radians.
    l1     : float
       Length of the link.
    
    Returns
    -------
    tcp : nd-array (x,y,z)
       The position of the tool center point (the origin of the tool frame) 
       in the base frame.
    theta : float
       The orientation of the end-effector with respect to the positive global x-axis.
       The angle returned is in the range [-np.pi, np.pi]
    R_ts  : nd-array (3x3)
       The rotation matrix
    J_s   : nd-array (3)
       The spatial jacobian
    
    Tests
    ------
    1) At reference position
    >>> l1 = 2.5
    >>> p, th, R, J = fwd_kinematics3(0, l1)
    >>> "x=%0.2f, y=%0.2f, z=%0.2f, th=%0.2f" %(p[0], p[1], p[2], th)
    'x=2.50, y=0.00, z=0.00, th=0.00'
    >>> np.any(np.abs(R - np.eye(3)) > 1e-12) # Rotation is identity
    False
    >>> np.any(np.abs(J - np.array([0., l1, 0. ])) > 1e-12) # Spatial vel upwards
    False
    
    2) At 90 degrees in joint
    >>> l1 = 2.5
    >>> p, th, R, J = fwd_kinematics3(np.pi/2, l1)
    >>> "x=%0.2f, y=%0.2f, z=%0.2f, th=%0.2f" %(p[0], p[1], p[2], th)
    'x=0.00, y=2.50, z=0.00, th=1.57'
    >>> np.any(np.abs(R - np.array([[0, 1.0, 0],[-1.0, 0, 0], [0,0,1]]))> 1e-12) # Rotation
    False
    >>> np.any(np.abs(J - np.array([-l1, 0., 0. ])) > 1e-12) # Spatial vel left
    False
    
    3) Compare spatial velocity with finite difference
    >>> l1 = 2.5
    >>> speed = 1.5 # rad/s
    >>> dt = 1e-8
    >>> delta_angle = speed*dt
    >>> p0, th0, R0, J0 = fwd_kinematics3(np.pi/3, l1)
    >>> p1, th1, R1, J1 = fwd_kinematics3(np.pi/3+delta_angle, l1)
    >>> v = (p1-p0)/dt
    >>> v0 = J0*speed
    >>> np.any(np.abs(v-v0) > 1e-6)
    False
    '''
    
    tcp0 = np.array([l1, 0, 0]) # TCP in base frame at ref config
    
    return (tcp, theta, Rts, Js)


### Run doctests
If tests pass, no output is generated.

In [78]:
doctest.run_docstring_examples(fwd_kinematics3, globals())